# Code for fitting Astro RC data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
sns.set_palette('colorblind',20)
sns.set_context('notebook')
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)
matplotlib.rc('axes',labelsize=15) 
import pandas as pd
import pystan

import os
import pickle

els = False

import sys

In [2]:
def read_data():
#     sfile = '../data/KepxDR2/rcxyu18.csv'
    sfile = '../Data/rcxyu18.csv'
    df = pd.read_csv(sfile,index_col=0)
    return df
    
def normal(x, mu, sigma):
    return (1/np.sqrt(2*np.pi*sigma**2)) * np.exp(-(x - mu)**2/(2*sigma**2))

In [3]:
df = read_data()
print('Size: '+str(len(df)))
df.head(2)

Size: 5578


,kepmag_x,Length_Quarters,Length_days,numax,numax_err,dnu,dnu_err,amplitude,err.2_x,fwhm,...,BC_GBP,BC_GRP,BC_J,BC_H,BC_K,ast_MKs,ast_MH,ast_MJ,ast_MG,ast_M_err
KICID,,,,,,,,,,,,,,,,,,,,,
892760,13.23,6,380.8,29.48,0.48,3.962,0.116,149.7,8.3,12.0,...,-0.740,0.388,1.206,1.638,1.766,-1.671046,-1.543046,-1.111046,0.343954,0.206767
1026084,12.14,15,1139.0,41.17,0.90,4.414,0.061,63.8,2.9,24.8,...,-0.754,0.409,1.252,1.706,1.840,-1.878376,-1.744376,-1.290376,0.205624,0.166498


## Stan model for astrometric values

In [4]:
overwrite = False
asterostan = '''
functions {
    real bailerjones_lpdf(real r, real L){
        return log((1/(2*L^3)) * (r*r) * exp(-r/L));
    }
}
data {
    int<lower = 0> N;
    real m[N];
    real<lower=0> m_err[N];
    real oo[N];
    real oo_err[N];
    real<lower=0> RlEbv[N];
    
    real mu_init;
    real mu_spread;
//    real sigma_init;
//    real sigma_spread;
}
parameters {
    //Hyperparameters
    real mu;
    real<lower=0.> sigma;
    real<lower=1.> sigo;
    real<lower=0.5,upper=1.> Q;
    real<lower=.1, upper=4000.> L;
    
    //Latent parameters
    real M_infd_std[N];
    real Ai[N];
    real<lower = 1.> r_infd[N];
    
    // Parallax offset
    real oo_zp;
}
transformed parameters{
    //Inferred and transformed parameters
    real M_infd[N];
    
    //Operations
    for (n in 1:N){
        M_infd[n] = mu + sigma * M_infd_std[n]; //Rescale the M fit
    }
}
model {
    //Define calculable properties
    real m_true[N];
    real oo_exp[N];
    
    //Hyperparameters in true space [p(theta_rc, L)]
    mu ~ normal(mu_init, mu_spread); // Prior from seismo
    sigma ~ normal(0., 1.); // Spread from seismo
    Q ~ normal(1., .25);
    sigo ~ normal(3.0, 1.0);
    L ~ uniform(0.1, 4000.);
    
    //Latent parameters [p(alpha_i | theta_rc, L)]
    Ai ~ normal(RlEbv, 0.03);
    for (n in 1:N){
        r_infd[n] ~ bailerjones(L);
        target += log_mix(Q,
            normal_lpdf(M_infd_std[n] | 0., 1.),
            normal_lpdf(M_infd_std[n] | 0., sigo));
    }
    
    //Calculable properties
    for (n in 1:N){
        m_true[n] = M_infd[n] + 5*log10(r_infd[n]) - 5 + Ai[n];
        oo_exp[n] = (1000./r_infd[n]) + oo_zp;
    }

    //Observables [p(D | theta_rc, L, alpha)]
    oo ~ normal(oo_exp, oo_err); //Measurement uncertainty on parallax
    m ~ normal(m_true, m_err); //Measurement uncertainty on magnitude
    
    //
    oo_zp ~ normal(0.0, 0.5); // Prior on the offset!
    
}

'''

model_path = 'astrostan.pkl'
if overwrite:
    print('Updating Stan model')
    sm = pystan.StanModel(model_code = asterostan, model_name='astrostan')
    with open(model_path, 'wb') as f:
        pickle.dump(sm, f)
else:
    if os.path.isfile(model_path):
        print('Loading in existing Stan Model') 
        sm = pickle.load(open(model_path, 'rb'))
    else:
        print('Saving Stan Model') 
        sm = pystan.StanModel(model_code = asterostan, model_name='astrostan')
        with open(model_path, 'wb') as f:
            pickle.dump(sm, f)
print('Completed')

Loading in existing Stan Model
Completed


#### Run the model

In [ ]:
sel = df[:1000]
m = sel.Kmag.values
m_err = sel.e_Kmag.values
oo = sel.parallax.values
oo_err = sel.parallax_error.values
RlEbv = sel.Aks.values

dat = {'N': len(sel),
        'm': m,              #Magnitude in given band
        'm_err' : m_err,     #Magnitude error
        'oo' : oo,       #Observed parallax
        'oo_err' : oo_err,   #Observed parallax error
        'RlEbv':RlEbv,   #Reddening
        
        'mu_init':-1.668,  #Priors from seismo
        'mu_spread':0.002}

initd = {'mu':-1.668,        #Red clump position
        'sigma':0.035,         #Red clump spread
        'Q': 0.88,           #Mixture model weighting factor
        'sigo':4.,          #Sprea dof the outlier population (as a fn of sigma)
        'L':1000.,
        'oo_zp':-0.3}        #Distance prior length scale

fit = sm.sampling(data=dat, iter=4000, chains=4, init=[initd,initd, initd, initd])

/home/davies/.local/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [ ]:
import corner
chain = np.array([fit['mu'],fit['sigma'],fit['Q'],fit['sigo'],fit['L'], fit['oo_zp']])
corner.corner(chain.T,\
                labels=[r'$\mu$(mag)',r'$\sigma$(mag)','Q',r'$\sigma_o$(mag)','L(pc)', 'Offset'],\
                quantiles=[0.16, 0.5, 0.84],\
                show_titles=True, title_kwargs={"fontsize": 15})
plt.show()

In [ ]:
fit.plot()
plt.show()

In [ ]:
print(fit)

In [ ]:
s = fit.summary()
rhat = s['summary'][:,-1]
rhatfin = rhat[np.isfinite(rhat)]
print('Total number of Rhats: '+str(len(rhat)))
print('Total number of Rhats with the NaN values removed: '+str(len(rhatfin)))
sns.distplot(rhatfin)
plt.show()

### Some further diagnostic plots

In [ ]:
mu = np.median(fit['mu'])
sigma = np.median(fit['sigma'])
sigo = (np.median(fit['sigo']))* sigma
Q = np.median(fit['Q'])
Minfd = np.median(fit['M_infd'],axis=0)
rinfd = np.median(fit['r_infd'],axis=0)
Linfd = np.median(fit['L'],axis=0)

x = np.linspace(Minfd.min(), Minfd.max(), 1000)
fg = normal(x, mu, sigma)
bg = normal(x, mu, sigo)
L = Q*fg + (1-Q)*bg

In [ ]:
left, bottom, width, height = 0.1, 0.47, 1., 0.60
fig = plt.figure(1, figsize=(8,8))
sax = fig.add_axes([left, bottom, width, height])
xax = fig.add_axes([left, 0.1, width, 0.3],sharex=sax)

sax.scatter(Minfd,m,s=5,zorder=1000)

sns.distplot(Minfd, ax=xax, label='M obs')
xax.plot(x,fg,label='Foreground',zorder=999)
xax.plot(x,bg,label='Background',zorder=998)
xax.plot(x, L,label='True likelihood',zorder=997)
xax.legend()

sax.grid()
xax.grid()

sax.set_xlabel('Absmag in K')
sax.set_ylabel('App mag in K')
xax.set_ylabel('Units arbitrary')

The above plot compares the inferred absolute Kband magnitude to the observed absolute Kband magnitude--- there are more data points at the red clump overdensity which is expected, as the points will have been placed there within the constraints of the observational uncertainties. 

There appears to be some kind of offset still, as there appears to be some correlation around the bisector at the inferred absolute magnitude positions.

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
f = np.abs(sel.parallax_error/sel.parallax)
c = plt.scatter(sel.r_est,rinfd,c=f, vmin=0., vmax=1.)
fig.colorbar(c, label='Fractional parallax uncertainty',extend='max')
ax.plot(sel.r_est, sel.r_est,c='k',linestyle='-.')
ax.set_xlabel('Inferred distance')
ax.set_ylabel('Bailer-Jones catalogue distance')
plt.show()
print('Our L: '+str(Linfd))
print('BJ+18 median L: '+str(np.median(sel.r_length_prior)))

Given that our L is smaller, we expect the mode of the prior distribution to be at a **lower distance**, and therefore we expect stars with high fractional uncertainties to be have a **lower inferred distance**. This means we would expect high fractional uncertainty stars to be *below* the bisector. That appears to be the case, although we'll need to run on more stars to confirm.

In [ ]:
r = sel.r_est
prior = (1/(2*Linfd**3)) * (r*r) * np.exp(-r/Linfd)

sns.distplot(sel.r_est,label='r est')
sns.distplot(rinfd, label='r infd')
plt.scatter(r,prior,s=5,label='Prior')
plt.legend()
plt.show()

From this it looks like the best fit value of L does not comply well for the full dataset--- could this be because L varies as a function of galactic position?

### Temporarily redundant code

In [ ]:
sys.exit()

In [ ]:
L = 1000.
r = np.linspace(0.1,8000, 100000)
p = (1/(2*L**3))*(r*r) * exp(-r/L)
print(p)
plt.plot(r, p)
plt.show()

In [ ]:
'''    //Hyperparameter Uniform priors for testing
        mu ~ uniform(-2., -1.);
        sigma ~ uniform(0.01, .15);
        Q ~ uniform(.8, 1.);
        muo ~ uniform(-2, -1.);
        sigo ~ normal(1.5, .5);
        L ~ uniform(0.1, 4000.);
'''

In [ ]:
plt.scatter(df.Ak_bj, df.Ak_pl)
plt.plot(df.Ak_bj,df.Ak_bj,label='Bisector',c='k',linestyle='-.')
plt.plot(df.Ak_bj,df.Ak_bj+0.05, label='+0.05mag',c='k',linestyle='-.')
plt.plot(df.Ak_bj,df.Ak_bj-0.05, label='-0.05mag',c='k',linestyle='-.')
plt.legend()
plt.xlabel('Extinction in K using Bailerjones')
plt.ylabel('Extinction in K using 1/parallax')
plt.show()

In [ ]:
'''Now lets quickly compare the inferred value of M and the omnitool value'''
fig, ax = plt.subplots(figsize=(8,8))
f = np.abs(sel.parallax_error/sel.parallax)
c= ax.scatter(sel.phot_MKs, Minfd,zorder=2,c=f,vmin=0.,vmax=1.)
fig.colorbar(c, label='Fractional parallax uncertainty',extend='max')
ax.errorbar(sel.phot_MKs, Minfd,xerr=sel.phot_MKs_err,c='grey',fmt='none',alpha=.3,zorder=1)
ax.plot(sel.phot_MKs, sel.phot_MKs, c='k',linestyle='-.')
ax.set_xlabel('Measured photometric absmag')
ax.set_ylabel('Inferred absmag')
plt.show()